In [1]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [2]:
from qiskit_optimization.runtime import QAOAClient
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp

In [3]:
import os
from qiskit import IBMQ
IBMQ.enable_account(os.environ["IBMQ_TOKEN"])

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
provider = IBMQ.get_provider(hub="ibm-q", group="open", project="main")


In [5]:
qaoa_mes = QAOAClient(
    provider=provider, backend=provider.get_backend("simulator_mps"), reps=2, alpha=0.75
)

qaoa = MinimumEigenOptimizer(qaoa_mes)

In [6]:
from src.quadratic_solver import quadratic_model, load_linear_prog_object
from src.utils import load_docpex_model

In [7]:
qubo = quadratic_model("lp_tiny.pkl")
linear = load_docpex_model("examples/tiny.lp")

In [8]:
len(list(qubo.iter_variables()))
qubo_qiskit = from_docplex_mp(qubo)
lin_qiskit = from_docplex_mp(linear)

In [9]:
lin_qiskit.get_num_vars()

10

In [10]:
result = qaoa.solve(lin_qiskit)
print(result.prettyprint())

RuntimeError: The job cdmdavi2h6kvivdih780 failed unexpectedly.

In [ ]:
print(provider.runtime.program("qaoa"))

qaoa:
  Name: qaoa
  Description: Qiskit Runtime QAOA program.
  Creation date: 2021-12-15T14:28:41.593027Z
  Update date: 2022-11-04T20:31:29.59236Z
  Max execution time: 18000
  Input parameters:
    Properties:
        - alpha:
            Description: The fraction of top measurement samples to be used for the expectation value (CVaR expectation). Defaults to 1, i.e. using all samples to construct the expectation value.
            Type: number
            Required: False
        - aux_operators:
            Description: A list of operators to be evaluated at the final, optimized state. This must be a List[PauliSumOp].
            Type: array
            Required: False
        - initial_point:
            Description: Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters. The type must be numpy.ndarray or str.
            Type: ['array', 'string']
            Required: False
        - measurement_error_mitigation:
         

In [8]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(token=os.environ["IBMQ_TOKEN"], auth="legacy")


/tmp/ipykernel_39026/848400363.py:2: DeprecationWarning: Use of `auth` parameter is deprecated and will be removed in a future release. You can now use channel='ibm_cloud' or channel='ibm_quantum' instead.
  service = QiskitRuntimeService(token=os.environ["IBMQ_TOKEN"], auth="legacy")


In [9]:
service.backends()

[<IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('ibmq_montreal')>,
 <IBMBackend('ibmq_toronto')>,
 <IBMBackend('ibmq_kolkata')>,
 <IBMBackend('ibmq_mumbai')>,
 <IBMBackend('simulator_statevector')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_hanoi')>,
 <IBMBackend('ibm_cairo')>,
 <IBMBackend('ibm_auckland')>,
 <IBMBackend('ibm_washington')>,
 <IBMBackend('ibm_geneva')>,
 <IBMBackend('ibmq_lima')>,
 <IBMBackend('ibmq_belem')>,
 <IBMBackend('ibmq_quito')>,
 <IBMBackend('ibmq_manila')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_oslo')>]

In [71]:
from scipy.sparse import load_npz
import io
from scipy.io import mmwrite


In [73]:
buffer = io.BytesIO()
J = load_npz("tiny_qubo_coo.npz")
mmwrite(buffer, J, field="real", symmetry="symmetric", precision=4)  